In [ ]:
The selected data files are quite large, so this data preprocessing was carried out on a local computer.

**Import libraries**


In [2]:
import sys
print(sys.executable)
import numpy as np
import xarray as xr
from pathlib import Path


D:\miniconda\envs\research\python.exe


**step.1 Read data files**

First, we read and checked the NetCDF dThe latitude and longitude variable names differ among files, so they need to be automatically detectedata files, including latitude/longitude coordinates and time range, to ensure the accuracy of subsequent analysis.

In [3]:
# Set path
data_dir = Path(r'D:\cangku\data')

# List of NetCDF files to analyze
files = [
    #gpp（abrupt-4xCO2，G1，piControl）
    'gpp_Lmon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012.nc',
    'gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912.nc',
    'gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912.nc',
    #tos（abrupt-4xCO2，G1，piControl）
    "tos_Omon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012.nc",
    "tos_Omon_CanESM5_G1_r1i1p2f1_gn_185001-194912.nc",
    "tos_Omon_CanESM5_piControl_r1i1p2f1_gn_185001-194912.nc",
    #tas（abrupt-4xCO2，G1，piControl）
    'tas_Amon_CanESM5_abrupt-4xCO2_185001-200012.nc',     
    'tas_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912.nc',
    #pr（abrupt-4xCO2，G1，piControl）
    'pr_Amon_CanESM5_abrupt-4xCO2_185001-200012.nc',   
    'pr_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912.nc',
]

In [4]:
# The latitude and longitude variable names differ among files, so they need to be automatically detected
def find_lat_lon_names(ds):
    lat_name, lon_name = None, None
    for candidate in ['lat', 'latitude', 'nav_lat', 'y']:
        if candidate in ds.variables:
            lat_name = candidate
            break
    for candidate in ['lon', 'longitude', 'nav_lon', 'x']:
        if candidate in ds.variables:
            lon_name = candidate
            break
    if lat_name is None or lon_name is None:
        raise ValueError('Failed to automatically identify latitude and longitude variable names')
    return lat_name, lon_name

In [5]:
# Check each file and print a short summary
def summarize_dataset(file_path):
    time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
    with xr.open_dataset(file_path, decode_times=time_coder) as ds:
        lat_name, lon_name = find_lat_lon_names(ds)    # Read different latitude and longitude names
       
        lat = ds[lat_name]
        lon = ds[lon_name]
        time = ds['time']

        print(f'File: {file_path.name}')
        print(f'  Latitude dimensions: {lat.shape}')
        print(f'  Longitude dimensions: {lon.shape}')
        print(f'  Time dimension length: {time.size}')
        print(f'  Time range: {str(time.values[0])} to {str(time.values[-1])}')
                
for filename in files:
    path = data_dir / filename
    if path.exists():
        try:
            summarize_dataset(path)
        except Exception:
            print(f'Failed to read {filename}. Please verify the file content.')
    else:
        print(f'{filename} not found in the data directory.')
        

File: gpp_Lmon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012.nc
  Latitude dimensions: (64,)
  Longitude dimensions: (128,)
  Time dimension length: 1812
  Time range: 1850-01-16 12:00:00 to 2000-12-16 12:00:00
File: gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912.nc
  Latitude dimensions: (64,)
  Longitude dimensions: (128,)
  Time dimension length: 1200
  Time range: 1850-01-16 12:00:00 to 1949-12-16 12:00:00
File: gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912.nc
  Latitude dimensions: (64,)
  Longitude dimensions: (128,)
  Time dimension length: 1200
  Time range: 1850-01-15 00:00:00 to 1949-12-15 00:00:00
File: tos_Omon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012.nc
  Latitude dimensions: (291, 360)
  Longitude dimensions: (291, 360)
  Time dimension length: 1812
  Time range: 1850-01-16 12:00:00 to 2000-12-16 12:00:00
File: tos_Omon_CanESM5_G1_r1i1p2f1_gn_185001-194912.nc
  Latitude dimensions: (291, 360)
  Longitude dimensions: (291, 360)
  Time dimension length: 1200
  

**step.2 Data processing **

2.1 Interpolation (unifying spatial resolution)

In [ ]:
Since the spatial resolutions of model outputs vary, all data were interpolated to a standard 1°×1° grid.

In [5]:
# Define interpolation grid (1° × 1°)
lon_target = np.arange(0, 360, 1) #The original longitude is 180°W–180°E, convert to 0–360 for easier calculation
lat_target = np.arange(-90, 90, 1)

In [99]:
#Ensure all datasets have consistent latitude/longitude naming and coordinate systems

def fix_lon(lon):
    return lon % 360  #Convert longitude to the 0–360 range

def standardize_coords(ds):
    # Standardize latitude and longitude variable names
    for lat_name in ['lat', 'latitude', 'nav_lat', 'y']:
        if lat_name in ds:
            ds = ds.rename({lat_name: 'lat'})
            break
    for lon_name in ['lon', 'longitude', 'nav_lon', 'x']:
        if lon_name in ds:
            ds = ds.rename({lon_name: 'lon'})
            break

    # tos data have 2D lat/lon, need to simplify to 1D
    if ds['lon'].ndim == 2 and ds['lat'].ndim == 2:
        lat_1d = ds['lat'][:, 0].values
        lon_1d = ds['lon'][0, :].values
        lon_1d = fix_lon(lon_1d)
        ds = ds.drop_vars(['lon', 'lat'], errors='ignore')
        ds = ds.rename({'j': 'lat', 'i': 'lon'})
        ds = ds.assign_coords(lon=lon_1d, lat=lat_1d)

    ds['lon'] = fix_lon(ds['lon'])
    return ds.sortby('lon')

In [7]:
#Define function for interpolating a single file
def regrid(file):
    with xr.open_dataset(file) as ds:
        var = file.name.split('_')[0]
        if var not in ds:
            var = list(ds.data_vars)[0] 

        ds = ds[[var, 'time']]
        ds = standardize_coords(ds)

        ds_interp = ds.interp(lon=lon_target, lat=lat_target, method='linear') #Use linear interpolation to unify resolution

        out_file = file.with_name(file.stem + '_1x1.nc')
        ds_interp.to_netcdf(out_file)  

        print(f'Finished: {out_file.name}')
        
#Execute interpolation
for f in files:
    file_path = data_dir / f
    if file_path.exists():
        try:
            regrid(file_path)
        except Exception as e:
            print(f'Error: {f}, {e}') 
    else:
        print(f'File not found: {f}')

Finished: gpp_Lmon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1.nc
Finished: gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc
Finished: gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1.nc
Finished: tos_Omon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1.nc
Finished: tos_Omon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc
Finished: tos_Omon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1.nc
Finished: tas_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1.nc
Finished: tas_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc
Finished: pr_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1.nc
Finished: pr_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc


**2.2 Detrend**

We performed detrending on the data to remove the effects of long-term external forcing (such as rising CO₂ or different model configurations) and to highlight the true response of terrestrial carbon fluxes to ENSO-related interannual climate variability.

In [3]:
input_files = [
    'gpp_Lmon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1.nc',
    'gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc',
    'gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1.nc',
    'tos_Omon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1.nc',
    'tos_Omon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc',
    'tos_Omon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1.nc',
    'tas_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1.nc',
    'tas_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc',
    'pr_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1.nc',
    'pr_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1.nc',
]

In [7]:
#Define detrending function
def detrend(data):
    t, nlat, nlon = data.shape
    time_index = np.arange(t)
    out = np.full_like(data, np.nan)

    for i in range(nlat):
        for j in range(nlon):
            y = data[:, i, j]
            if np.isfinite(y).sum() < 2:
                continue
            p = np.polyfit(time_index, y, 1) #Least squares method
            out[:, i, j] = y - np.polyval(p, time_index)
    return out

def process_one(path):
    with xr.open_dataset(path) as ds:
        var = next((v for v in ['gpp', 'tos', 'tas', 'pr'] if v in ds), None)
        if not var:
            print(f'Skipped {path.name}, main variable not found.')
            return
        da = ds[var].sel(time=slice('1890-01', '1949-12')).astype('float64') # Detrending period: 1890–1949
        detrended = detrend(da.values)

        da_dt = xr.DataArray(
            detrended, coords=da.coords, dims=da.dims, name=var, attrs=da.attrs
        )

        out_path = path.with_name(path.stem + '_1890-1949_detrend1.nc')
        da_dt.to_netcdf(out_path)
        print(f'Finished: {out_path.name}')

# Batch detrending
for fname in input_files:
    path = data_dir / fname
    if path.exists():
        try:
            process_one(path)
        except Exception as e:
            print(f'Error processing {fname}: {e}')
    else:
        print(f'File not found: {fname}')

Finished: gpp_Lmon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1_1890-1949_detrend1.nc
Finished: gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend1.nc
Finished: gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend1.nc
Finished: tos_Omon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1_1890-1949_detrend1.nc
Finished: tos_Omon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend1.nc
Finished: tos_Omon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend1.nc
Finished: tas_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1_1890-1949_detrend1.nc
Finished: tas_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend1.nc
Finished: pr_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1_1890-1949_detrend1.nc
Finished: pr_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend1.nc


In [4]:
# Read all files, crop latitude (40° for others; ±5° for tos), and convert to float32
files = sorted(data_dir.glob("*_detrend1.nc"))

def crop_and_compress(path):
    try:
        with xr.open_dataset(path) as ds:
            # Standardize latitude and longitude variable names
            lat_name = None
            for name in ["lat", "latitude", "nav_lat"]:
                if name in ds.coords:
                    lat_name = name
                    break

            # Determine cropping range: 40° for most variables, ±5° for tos
            if "tos" in path.name:
                lat_range = slice(-5, 5)
            else:
                lat_range = slice(-40, 40)

            # Apply cropping only if latitude is 1D
            if lat_name and ds[lat_name].ndim == 1:
                ds = ds.sel({lat_name: lat_range})
                print(f"{path.name}: Latitude cropped to {lat_range.start}° ~ {lat_range.stop}°")
            else:
                print(f"{path.name}: Unable to crop (latitude is 2D or missing), compression only")

            # Convert to float32 (reduce storage size and make computation faster)
            ds32 = ds.astype("float32")

           
            new_name = path.name.replace("detrend1.nc", "detrend.nc")
            out_path = path.with_name(new_name)

            # Set compression parameters
            encoding = {
                var: {"zlib": True, "complevel": 4, "dtype": "f4"}
                for var in ds32.data_vars
            }

            ds32.to_netcdf(out_path, encoding=encoding)
            print(f"Saved: {out_path.name}\n")

    except Exception as e:
        print(f"Error processing {path.name}: {e}\n")


# === Batch execution ===
for f in files:
    crop_and_compress(f)


gpp_Lmon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1_1890-1949_detrend1.nc: Latitude cropped to -40° ~ 40°
Saved: gpp_Lmon_CanESM5_abrupt-4xCO2_r1i1p2f1_gn_185001-200012_1x1_1890-1949_detrend.nc

gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend1.nc: Latitude cropped to -40° ~ 40°
Saved: gpp_Lmon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend.nc

gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend1.nc: Latitude cropped to -40° ~ 40°
Saved: gpp_Lmon_CanESM5_piControl_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend.nc

pr_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1_1890-1949_detrend1.nc: Latitude cropped to -40° ~ 40°
Saved: pr_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1_1890-1949_detrend.nc

pr_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend1.nc: Latitude cropped to -40° ~ 40°
Saved: pr_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend.nc

tas_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1_1890-1949_d

In [3]:
#Convert monthly temperature (tas) and precipitation (pr) data to annual mean values (1890–1949)

DATA_DIR = Path(r'D:\cangku\data')

TAS_FILES = {
    'abrupt-4xCO2': 'tas_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1_1890-1949_detrend.nc',
    'G1': 'tas_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend.nc',
}

PR_FILES = {
    'abrupt-4xCO2': 'pr_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1_1890-1949_detrend.nc',
    'G1': 'pr_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend.nc',
}
LAT_SLICE = slice(-40, 40)


def _std_coords(ds):
    if 'latitude' in ds.coords:
        ds = ds.rename({'latitude': 'lat'})
    if 'longitude' in ds.coords:
        ds = ds.rename({'longitude': 'lon'})
    if 'lon' in ds.coords and float(ds['lon'].min()) < 0:
        ds = ds.assign_coords(lon=(ds['lon'] % 360)).sortby('lon')
    return ds

def compute_annual_mean_field(path: Path, varname: str):   
    ds = xr.open_dataset(path)
    ds = _std_coords(ds)

    # Identify Variable
    if varname not in ds.data_vars:
        varname = list(ds.data_vars)[0]
    da = ds[varname].sel(lat=LAT_SLICE)

    # convert to annual mean
    annual = da.resample(time='YS').mean().sel(time=slice('1890', '1949'))
    annual = annual.astype('float32')

    ds.close()
    return annual


def process_and_save(file_dict, varname):
    for exp, fname in file_dict.items():
        in_path = DATA_DIR / fname
        da_ann = compute_annual_mean_field(in_path, varname)
        out_name = fname.replace('.nc', '_annual_1890-1949.nc')
        out_path = DATA_DIR / out_name
        da_ann.to_netcdf(out_path)
        print(f'[finish] {varname} annual mean saved → {out_path.name}, shape={da_ann.shape}')


if __name__ == '__main__':
    process_and_save(TAS_FILES, 'tas')
    process_and_save(PR_FILES, 'pr')
    print(' all finished ')


[finish] tas annual mean saved → tas_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1_1890-1949_detrend_annual_1890-1949.nc, shape=(60, 81, 360)
[finish] tas annual mean saved → tas_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend_annual_1890-1949.nc, shape=(60, 81, 360)
[finish] pr annual mean saved → pr_Amon_CanESM5_abrupt-4xCO2_185001-200012_1x1_1890-1949_detrend_annual_1890-1949.nc, shape=(60, 81, 360)
[finish] pr annual mean saved → pr_Amon_CanESM5_G1_r1i1p2f1_gn_185001-194912_1x1_1890-1949_detrend_annual_1890-1949.nc, shape=(60, 81, 360)
=== all finished ===
